In [1]:
import os
os.environ["MANTLE"] = "coreir"
import silica as si
import magma as m
from silica import bits, add, uint
from magma.bit_vector import BitVector
from magma.testing.coroutine import check

@si.coroutine(inputs={"I" : si.Array(4, si.Bits(16))})
def Serializer4():
    data = [bits(0, 16) for _ in range(3)]
    I = yield
    while True:
        O = I[0]
        for i in range(3):
            data[i] = I[i + 1]
        I = yield O
        for i in range(3):
            O = data[i]
            I = yield O

        
serializer4 = si.compile(Serializer4(), file_name="build/silica_serializer4.py")
with open("build/silica_serializer4.py", "r") as magma_file:
    print(magma_file.read())

from magma import *
import os
os.environ["MANTLE"] = os.getenv("MANTLE", "coreir")
from mantle import *
import mantle.common.operator


@cache_definition
def DefineSilicaMux(height, width):
    if "one-hot" == "one-hot":
        if width is None:
            T = Bit
        else:
            T = Bits(width)
        inputs = []
        for i in range(height):
            inputs += [f"I{i}", In(T)]
        class OneHotMux(Circuit):
            name = "SilicaOneHotMux{}{}".format(height, width)
            IO = inputs + ["S", In(Bits(height)), "O", Out(T)]
            @classmethod
            def definition(io):
                or_ = Or(height, width)
                wire(io.O, or_.O)
                for i in range(height):
                    and_ = And(2, width)
                    wire(and_.I0, getattr(io, f"I{i}"))
                    if width is not None:
                        for j in range(width):
                            wire(and_.I1[j], io.S[i])
                    else:
   

In [2]:
print(repr(serializer4))

Serializer4 = DefineCircuit("Serializer4", "O", Out(Bits(16)), "I", Array(4,In(Bits(16))), "CLK", In(Clock))
inst0 = __silica_BufferSerializer4()
inst1 = Register5_0001()
inst2 = or_wrapped()
inst3 = Register16CE()
inst4 = Register16CE()
inst5 = Register16CE()
inst6 = SilicaOneHotMux516()
inst7 = Mux2x16()
inst8 = Mux2x16()
inst9 = Mux2x16()
inst10 = or_wrapped()
inst11 = or_wrapped()
inst12 = or_wrapped()
wire(inst1.O, inst0.I)
wire(0, inst1.I[0])
wire(inst2.O, inst1.I[1])
wire(inst1.O[1], inst1.I[2])
wire(inst1.O[2], inst1.I[3])
wire(inst1.O[3], inst1.I[4])
wire(None, inst1.CLK)
wire(inst1.O[0], inst2.I0)
wire(inst1.O[4], inst2.I1)
wire(inst7.O, inst3.I)
wire(None, inst3.CLK)
wire(inst10.O, inst3.CE)
wire(inst8.O, inst4.I)
wire(None, inst4.CLK)
wire(inst11.O, inst4.CE)
wire(inst9.O, inst5.I)
wire(None, inst5.CLK)
wire(inst12.O, inst5.CE)
wire(Serializer4.I[0], inst6.I0)
wire(inst3.O, inst6.I1)
wire(inst4.O, inst6.I2)
wire(inst5.O, inst6.I3)
wire(Serializer4.I[0], inst6.I4)
wire(inst0

In [4]:
ser = Serializer4()
inputs = [[4,5,6,7],[10,16,8,3]]
for I in inputs:
  ser.send(I)
  print(I, ser.O)
  for i in range(3):
    assert ser.O == I[i]
    next(ser)
    print(I, ser.O)
    

@si.coroutine
def inputs_generator(inputs):
    while True:
        for i in inputs:
            I = [BitVector(x, 16) for x in i]
            yield I
            for _ in range(3):
                I = [BitVector((_ * len(i)) + j, 16) for j in range(len(i))]
                yield I
                
check(serializer4, Serializer4(), 9, inputs_generator(inputs))
print("Passed!")

[4, 5, 6, 7] 4
[4, 5, 6, 7] 5
[4, 5, 6, 7] 6
[4, 5, 6, 7] 7
[10, 16, 8, 3] 10
[10, 16, 8, 3] 16
[10, 16, 8, 3] 8
[10, 16, 8, 3] 3
Passed!


In [3]:
!magma -o coreir -m coreir -t Serializer4 build/silica_serializer4.py
!coreir -i build/silica_serializer4.json -o build/silica_serializer4.v
# !magma -o verilog -m lattice -t Serializer4 build/silica_serializer4.py
!yosys -p 'synth_ice40 -top Serializer4 -blif build/silica_serializer4.blif' build/silica_serializer4.v  | grep -A 14 "2.27. Printing statistics."
!arachne-pnr -d 1k -o build/silica_serializer4.txt build/silica_serializer4.blif
!icetime -tmd hx1k build/silica_serializer4.txt  | grep -B 2 "Total path delay"

In Run Generators
ALREADY ADDED CONNECTION!
ALREADY ADDED CONNECTION!
ALREADY ADDED CONNECTION!
ALREADY ADDED CONNECTION!
ALREADY ADDED CONNECTION!
ALREADY ADDED CONNECTION!
ALREADY ADDED CONNECTION!
ALREADY ADDED CONNECTION!
ALREADY ADDED CONNECTION!
ALREADY ADDED CONNECTION!
ALREADY ADDED CONNECTION!
ALREADY ADDED CONNECTION!
ALREADY ADDED CONNECTION!
ALREADY ADDED CONNECTION!
ALREADY ADDED CONNECTION!
ALREADY ADDED CONNECTION!
ALREADY ADDED CONNECTION!
ALREADY ADDED CONNECTION!
ALREADY ADDED CONNECTION!
ALREADY ADDED CONNECTION!
ALREADY ADDED CONNECTION!
ALREADY ADDED CONNECTION!
ALREADY ADDED CONNECTION!
ALREADY ADDED CONNECTION!
ALREADY ADDED CONNECTION!
ALREADY ADDED CONNECTION!
ALREADY ADDED CONNECTION!
ALREADY ADDED CONNECTION!
ALREADY ADDED CONNECTION!
ALREADY ADDED CONNECTION!

Modified?: Yes
2.27. Printing statistics.

=== Serializer4 ===

   Number of wires:               2482
   Number of wire bits:           3953
   Number of public wires:        2449
   Number of public 

In [4]:
!yosys -p 'synth_ice40 -top serializer -blif build/verilog_serializer.blif' ../verilog/serializer.v | grep -A 14 "2.27. Printing statistics."
!arachne-pnr -d 1k -o build/verilog_serializer.txt build/verilog_serializer.blif
!icetime -tmd hx1k build/verilog_serializer.txt | grep -B 2 "Total path delay"

2.27. Printing statistics.

=== serializer ===

   Number of wires:                 27
   Number of wire bits:            151
   Number of public wires:           7
   Number of public wire bits:     131
   Number of memories:               0
   Number of memory bits:            0
   Number of processes:              0
   Number of cells:                 86
     SB_DFFE                        48
     SB_DFFSR                        2
     SB_LUT4                        36
seed: 1
device: 1k
read_chipdb +/share/arachne-pnr/chipdb-1k.bin...
  supported packages: cb121, cb132, cb81, cm121, cm36, cm49, cm81, qn84, swg16tr, tq144, vq100
read_blif build/verilog_serializer.blif...
prune...
instantiate_io...
pack...

After packing:
IOs          81 / 96
GBs          0 / 8
  GB_IOs     0 / 8
LCs          84 / 1280
  DFF        50
  CARRY      0
  CARRY, DFF 0
  DFF PASS   48
  CARRY PASS 0
BRAMs        0 / 16
WARMBOOTs    0 / 1
PLLs         0 / 1

place_constraints...
promote_globals...
  promot

In [5]:
import magma
from magma.testing.verilator import compile as compileverilator
from magma.testing.verilator import run_verilator_test
def check_verilog(circ, circ_sim, num_cycles, inputs_generator=None):
    test_vectors = magma.testing.coroutine.testvectors(circ, circ_sim, num_cycles,
            inputs_generator if inputs_generator else None)

    name = "serializer"
    with open(f"build/sim_{name}_verilator.cpp", "w") as verilator_harness:
        verilator_harness.write(f'''\
#include "V{name}.h"
#include "verilated.h"
#include <cassert>
#include <iostream>

void check(const char* port, int a, int b, int cycle) {{
    if (!(a == b)) {{
        std::cerr << \"Got      : \" << a << std::endl;
        std::cerr << \"Expected : \" << b << std::endl;
        std::cerr << \"Cycle    : \" << cycle << std::endl;
        std::cerr << \"Port     : \" << port << std::endl;
        exit(1);
    }}
}}

int main(int argc, char **argv, char **env) {{
    Verilated::commandArgs(argc, argv);
    V{name}* top = new V{name};
''')
    
        cycle = 0
        for inputs, outputs in zip(test_vectors[0], test_vectors[1]):
            print(inputs) 
            print(outputs)
            for port_name, value in inputs.items():
                if isinstance(value, list):
                    string = ""
                    for elem in value:
                        string = elem.as_binary_string()[2:] + string
                    value = "0b" + string
                else:
                    value = int(value)
                verilator_harness.write("    top->{} = {};\n".format(port_name, value))
            for port_name, value in outputs.items():
                if isinstance(value, BitVector):
                    value = value.as_binary_string()
                else:
                    value = int(value)
                verilator_harness.write("    top->eval();\n")
                verilator_harness.write("    check(\"{port_name}\", top->{port_name}, {expected}, {cycle});\n".format(port_name=port_name, expected=value, cycle=cycle))

            verilator_harness.write("    top->CLK = 0;\n")
            verilator_harness.write("    top->eval();\n")
            verilator_harness.write("    top->CLK = 1;\n")
            verilator_harness.write("    top->eval();\n")
            cycle += 1
        verilator_harness.write("}\n")

    run_verilator_test(
        "serializer",
        f"sim_{name}_verilator",
        name,
        "-I../../verilog"
    )



check_verilog(serializer4, Serializer4(), 9, inputs_generator(inputs))
print("Passed!")

{'I': [bits(4, 16), bits(5, 16), bits(6, 16), bits(7, 16)]}
{'O': bits(4, 16)}
{'I': [bits(0, 16), bits(1, 16), bits(2, 16), bits(3, 16)]}
{'O': bits(5, 16)}
{'I': [bits(4, 16), bits(5, 16), bits(6, 16), bits(7, 16)]}
{'O': bits(6, 16)}
{'I': [bits(8, 16), bits(9, 16), bits(10, 16), bits(11, 16)]}
{'O': bits(7, 16)}
{'I': [bits(10, 16), bits(16, 16), bits(8, 16), bits(3, 16)]}
{'O': bits(10, 16)}
{'I': [bits(0, 16), bits(1, 16), bits(2, 16), bits(3, 16)]}
{'O': bits(16, 16)}
{'I': [bits(4, 16), bits(5, 16), bits(6, 16), bits(7, 16)]}
{'O': bits(8, 16)}
{'I': [bits(8, 16), bits(9, 16), bits(10, 16), bits(11, 16)]}
{'O': bits(3, 16)}
{'I': [bits(4, 16), bits(5, 16), bits(6, 16), bits(7, 16)]}
{'O': bits(4, 16)}
Passed!
